In [2]:
import pandas as pd
import re
import os
import sys
sys.path.append('/home3/phungqv/number_normalizer/vietnam_number')
import vietnam_number

In [149]:
# re.sub(number, add_space_to_num, '2 đô la sự việc xảy phẩy ra vào cộng ngày a còng 9 2 chấm 239847,30294 asdfsdas 239048.5409385')

## Process data (numbers not converted to word)

In [150]:
d = {'input': [], 'target' : []}
punctuation = '!"#&\',-:;?[\]^_`{|}().'

number = re.compile(r'(0|1|2|3|4|5|6|7|7|8|9){2,}')
def add_space_to_num(match):
    str_l = [char for char in match.group(0)]
    return ' '.join(str_l)

with open('/home3/phungqv/data_generate/data/100k/data_small.txt.norm') as data:
# data = ['2$ Sự việc xảy, ra vào + ngày@ 9/2 1098.210  32478,1238.	2$ Sự việc xảy, ra vào + ngày@ mùng chín tháng hai .']
    for line in data:
        line = line.lower()
        if '/' in  line:
            continue
        line = line.split('	')

        for i in range(2):
            line[i] = line[i].strip()
            line[i] = line[i].replace('$', ' đô la ')
            line[i] = line[i].replace(',', ' phẩy ')
            line[i] = line[i].replace('%', ' phần trăm ')
            line[i] = line[i].replace('@', ' a còng ')
            line[i] = line[i].replace('+' , ' cộng ')

            # line[i] = re.sub(number, add_space_to_num, line[i])      #for separating numbers

            line[i] = line[i].translate(str.maketrans(punctuation, ' '*len(punctuation)))
            line[i] = ' '.join(line[i].split())

        # line[0] = line[0].strip()
        # line[1] = line[1].strip()

        # line[0] = line[0].translate(str.maketrans(punctuation, ' '*len(punctuation)))
        # line[1] = line[1].translate(str.maketrans(punctuation, ' '*len(punctuation)))

        # line[0] = line[0].replace('$', ' đô la')


        # line[0] = ' '.join(line[0].split())
        # line[1] = ' '.join(line[1].split())

        d['input'].append(line[1])
        d['target'].append(line[0])

## Process data (numbers in labels converted to words like 20 -> hai mươi)

In [152]:
d = {'input': [], 'target' : []}
punctuation = '!"#&\',-:;?[\]^_`{|}().'

number = re.compile(r' (0|1|2|3|4|5|6|7|7|8|9)+')
def num_to_word(match):
    result = vietnam_number.n2w(match.group(0).strip())
    return result.replace('lẽ', 'linh')

with open('/home3/phungqv/data_generate/data/100k/data_small.txt.norm') as data:
# data = ['2$ Sự việc xảy, ra vào + ngày@ 9/2 1098.210  32478,1238.	2$ Sự việc xảy, ra vào + ngày@ mùng chín tháng hai .']
    for line in data:
        if '/' in line:
            continue
        line = line.lower()
        line = line.split('	')

        for i in range(2):
            line[i] = ' ' + line[i] + ' '
            line[i] = line[i].replace('$', ' đô la ')
            line[i] = line[i].replace(',', ' phẩy ')
            line[i] = line[i].replace('%', ' phần trăm ')
            line[i] = line[i].replace('@', ' a còng ')
            line[i] = line[i].replace('+' , ' cộng ')

            line[i] = re.sub(number, num_to_word, line[i])      #for converting numbers

            line[i] = line[i].translate(str.maketrans(punctuation, ' '*len(punctuation)))
            line[i] = line[i].strip()
            line[i] = ' '.join(line[i].split())

        d['input'].append(line[1])
        d['target'].append(line[0])

## Process data (words in labels converted to numbers like một trăm -> 100)

In [3]:
pattern_year_with_month = re.compile(r'((ngày|mùng)\s)?(?(1)((mười|mươi|một|mốt|hai|ba|bốn|tư|năm|lăm|sáu|bảy|bẩy|tám|chín)(\s)?){1,2})(tháng\s)(?(6)((một|hai|ba|tư|bốn|năm|sáu|bảy|bẩy|tám|chín|mười|mười một|mười hai)\s))năm', re.IGNORECASE)

pattern_num = re.compile(r'((không\s?)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)*(?!#)', re.IGNORECASE)

pattern_more1num = re.compile(r'((không\s)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)+', re.IGNORECASE)

pattern_year_only_full_num = re.compile(r'(một|hai|ba|bốn|năm|sáu|bảy|bẩy|tám|chín|mười)\s(mốt|lăm|tư|mươi|linh|lẻ|chục|trăm|nghìn|triệu|tỉ|tỷ)', re.IGNORECASE)

pattern_num_with_phay = re.compile(r'((không\s)*(một|hai|ba|bốn|tư|năm|sáu|bảy|bẩy|tám|chín|mười)\s?)((linh|lẻ|không|một|mốt|hai|ba|bốn|tư|năm|lăm|nhăm|sáu|bảy|bẩy|tám|chín|mười|mươi|chục|trăm|nghìn|ngàn|triệu|tỉ|tỷ)\s?)*,((không|một|hai|ba|bốn|năm|sáu|bảy|bẩy|tám|chín)\s?)+', re.IGNORECASE)


full_num_word_list = ['mươi', 'mười','linh','lẻ','chục','trăm','nghìn','ngàn','triệu','tỉ','tỷ','lăm','nhăm']

def check_single_num(num):
    for word in full_num_word_list:
        if word in num:
            return False
    return True


def num_with_phay_process(match):
    whole_num = match.group(0)
    num_list = whole_num.strip().split(',')
    new_num_list = []
    for num in num_list:
        if check_single_num(num):
            new_num_list.append(str(w2n_single(num)))
        else:
            new_num_list.append(str(w2n(num)))
    return ','.join(new_num_list) + ' '


def num_process(match):
    whole_num = match.group(0)
    year = ''
    if whole_num.startswith('năm'):
        if check_year_only_full_num(whole_num):
            year = '##' + whole_num[:3] + '##' + ' ' 
            whole_num = whole_num[3:]
    # else:
    if check_single_num(whole_num):
        return year + str(w2n_single(whole_num)) + ' '
    else:
        return year + str(w2n(whole_num)) + ' '


def year_with_month_num_process(match):
    num = match.group(0).strip()
    if check_single_num(num):
        return str(w2n_single(num)) + ' '
    else:
        return str(w2n(num)) + ' '

def year_with_month_process(match):
    result = match.group(0)[:-3] + '##' + match.group(0)[-3:] + '##'
    return re.sub(pattern_num, year_with_month_num_process, result)
    

def check_year_only_full_num(s):
    if not s.startswith('năm'):
        # print('doesnt start with nam')
        return False
    remove_nam = s[4:]
    for word in full_num_word_list:
        if remove_nam.startswith(word):
            return False
    result = re.search(pattern_year_only_full_num, s)
    if result:
        return True
    return False


number = re.compile(r'(0|1|2|3|4|5|6|7|7|8|9){2,}')

def add_space_to_num(match):
    str_l = [char for char in match.group(0)]
    return ' '.join(str_l)


def convert_num(sentence):
    sentence = re.sub(pattern_year_with_month, year_with_month_process, sentence)
    sentence = re.sub(pattern_num_with_phay, num_with_phay_process, sentence)
    # print(sentence)
    sentence = re.sub(pattern_more1num, num_process, sentence)
    sentence = sentence.replace('##', '')
    sentence = re.sub(number, add_space_to_num, sentence)
    
    return sentence

In [ ]:
d = {'input': [], 'target' : []}
punctuation = '!"#&\',-:;?[\]^_`{|}().'


# with open('/home3/phungqv/data_generate/data/100k/data_small.txt.norm') as data:
data = ['2$ Sự việc xảy, ra vào + ngày@ 9/2 1098.210  32478,1238.	2$ Sự việc xảy, ra vào + ngày@ mùng chín tháng hai .']
if data:
    for line in data:
        if '/' in line:
            continue
        line = line.lower()
        line = line.split('	')

        for i in range(2):
            line[i] = ' ' + line[i] + ' '
            line[i] = line[i].replace('$', ' đô la ')
            line[i] = line[i].replace(',', ' phẩy ')
            line[i] = line[i].replace('%', ' phần trăm ')
            line[i] = line[i].replace('@', ' a còng ')
            line[i] = line[i].replace('+' , ' cộng ')

            line[i] =  convert_num(sentence)
            line[i] = line[i].translate(str.maketrans(punctuation, ' '*len(punctuation)))
            line[i] = line[i].strip()
            line[i] = ' '.join(line[i].split())

        d['input'].append(line[1])
        d['target'].append(line[0])

In [202]:
punctuation = '!"#&\',-:;?[\]^_`{|}().'

number = re.compile(r' ((0|1|2|3|4|5|6|7|7|8|9)+\.?)+')
def num_to_word(match):
    print(match.group(0))
    result = vietnam_number.n2w(match.group(0).strip())
    return result.replace('lẽ', 'linh')
re.sub(number, num_to_word, ' 2000000')

 2000000


'hai triệu không trăm nghìn không trăm'

In [153]:
d['input'][0]

'có lẽ đó là một ngôi mộ tập thể'

In [154]:
df = pd.DataFrame(d)

In [155]:
len(df)

89574

In [156]:
df.iloc[-1]

input     nhóm cướp tiền phẩy vàng ở phú yên có ý định t...
target    nhóm cướp tiền phẩy vàng ở phú yên có ý định t...
Name: 89573, dtype: object

In [157]:
path = '/home3/phungqv/data_generate/data/100k/number_converted'
if not os.path.exists(os.path.join(path, 'data_small.csv')):
    print('exported')
    df.to_csv(os.path.join(path, 'data_small.csv'), index=False)

exported


In [158]:
df = pd.read_csv(os.path.join(path, 'data_small.csv'))


In [159]:
df.head()

,input,target
0,có lẽ đó là một ngôi mộ tập thể,có lẽ đó là một ngôi mộ tập thể
1,a xê an hy vọng sẽ đạt được bộ quy tắc ứng xử ...,asean hy vọng sẽ đạt được bộ quy tắc ứng xử bi...
2,cả nước chỉ có hai trăm ba mươi thanh tra y tế,cả nước chỉ có hai trăm ba mươi thanh tra y tế
3,khi đó phẩy em và bạn chạy theo phía sau bị hơ...,khi đó phẩy em và bạn chạy theo phía sau bị hơ...
4,mai mai theo x ta pun xờ,mai mai theo star pulse


In [161]:
added_errors = pd.read_json(os.path.join(path, 'train.json'))

In [162]:
added_errors.head()

,tid,index,input,output,error
0,0,0,"[có, nẽ, đó, nà, một, ngôi, mô, bập, thể]","[có, lẽ, đó, là, một, ngôi, mộ, tập, thể]","[0, 9, 0, 9, 0, 0, 15, 16, 0]"
1,1,1,"[a, pê, an, hy, vọng, sẽ, đạt, đườc, bộ, quy, ...","[a, xê, an, hy, vọng, sẽ, đạt, được, bộ, quy, ...","[0, 16, 0, 0, 0, 0, 0, 15, 0, 0, 15, 0, 0, 0, ..."
2,2,2,"[cả, nước, chỉ, có, haỉ, trăm, ba, mươi, thanh...","[cả, nước, chỉ, có, hai, trăm, ba, mươi, thanh...","[0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0]"
3,3,3,"[khi, đó, phẩy, em, và, bạn, trạy, theo, phía,...","[khi, đó, phẩy, em, và, bạn, chạy, theo, phía,...","[0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,4,"[mai, maì, theo, x, ta, pun, xờ]","[mai, mai, theo, x, ta, pun, xờ]","[0, 15, 0, 0, 0, 0, 0]"


In [163]:
del added_errors['tid']
del added_errors['index']
del added_errors['error']


In [164]:
added_errors.head()

,input,output
0,"[có, nẽ, đó, nà, một, ngôi, mô, bập, thể]","[có, lẽ, đó, là, một, ngôi, mộ, tập, thể]"
1,"[a, pê, an, hy, vọng, sẽ, đạt, đườc, bộ, quy, ...","[a, xê, an, hy, vọng, sẽ, đạt, được, bộ, quy, ..."
2,"[cả, nước, chỉ, có, haỉ, trăm, ba, mươi, thanh...","[cả, nước, chỉ, có, hai, trăm, ba, mươi, thanh..."
3,"[khi, đó, phẩy, em, và, bạn, trạy, theo, phía,...","[khi, đó, phẩy, em, và, bạn, chạy, theo, phía,..."
4,"[mai, maì, theo, x, ta, pun, xờ]","[mai, mai, theo, x, ta, pun, xờ]"


In [165]:
added_errors_str = added_errors.applymap(lambda l : ' '.join(l))

In [166]:
added_errors_str.tail()

,input,output
89569,ảnh the sun người đàn ông chạy thẳng qua chỗ đ...,ảnh the sun người đàn ông chạy thẳng qua chỗ đ...
89570,cán bộ ngân hàng giúp công ty ở bạc liêu lừa đảo,cán bộ ngân hàng giúp công ty ở bạc liêu lừa đảo
89571,danh sách các loại ma tuý mới không dừng lại p...,danh sách các loại ma tuý mới không dừng lại p...
89572,gia đình đã đưa chị đến điều trị tại bệnh viện...,gia đình đã đưa chị đến điều trị tại bệnh viện...
89573,nhóm cướp tiền phẩy vàng ở phú yên có ý định t...,nhóm cướp tiền phẩy vàng ở phú yên có ý định t...


In [167]:
final_df = pd.DataFrame()

In [168]:
final_df['input'] = added_errors_str.input
final_df['target'] = df.target

In [169]:
len(final_df)

89574

In [170]:
final_df.to_csv(os.path.join(path, 'final_data.csv'), index=False)

In [171]:
final_df = pd.read_csv(os.path.join(path, 'final_data.csv'))

In [172]:
final_df.tail()

,input,target
89569,ảnh the sun người đàn ông chạy thẳng qua chỗ đ...,ảnh the sun người đàn ông chạy thẳng qua chỗ đ...
89570,cán bộ ngân hàng giúp công ty ở bạc liêu lừa đảo,cán bộ ngân hàng giúp công ty ở bạc liêu lừa đảo
89571,danh sách các loại ma tuý mới không dừng lại p...,danh sách các loại ma túy mới không dừng lại p...
89572,gia đình đã đưa chị đến điều trị tại bệnh viện...,gia đình đã đưa chị đến điều trị tại bệnh viện...
89573,nhóm cướp tiền phẩy vàng ở phú yên có ý định t...,nhóm cướp tiền phẩy vàng ở phú yên có ý định t...


In [173]:
num_sentences = len(final_df)
val = final_df.iloc[0:4096]
train = final_df.iloc[4096:]

In [174]:
val.head()

,input,target
0,có nẽ đó nà một ngôi mô bập thể,có lẽ đó là một ngôi mộ tập thể
1,a pê an hy vọng sẽ đạt đườc bộ quy tăc ứng xử ...,asean hy vọng sẽ đạt được bộ quy tắc ứng xử bi...
2,cả nước chỉ có haỉ trăm ba mươi thanh tra y tế,cả nước chỉ có hai trăm ba mươi thanh tra y tế
3,khi đó phẩy em và bạn trạy theo phía sau bị hơ...,khi đó phẩy em và bạn chạy theo phía sau bị hơ...
4,mai maì theo x ta pun xờ,mai mai theo star pulse


In [175]:
train.head()

,input,target
4096,xuyến đã soạn thảo các thủ tục để xuất lậu một...,xuyến đã soạn thảo các thủ tục để xuất lậu một...
4097,đây nà loại máy bay ném bom chiền lược phẩy ba...,đây là loại máy bay ném bom chiến lược phẩy ba...
4098,đúng lúc đó phẩy xe buýt mang bê ca ét x hai m...,đúng lúc đó phẩy xe buýt mang bks hai mươi chí...
4099,đọ súng ác liệt đã nổ ra trong vụ vây ráp phây...,đọ súng ác liệt đã nổ ra trong vụ vây ráp phẩy...
4100,mhồng cu bã huện,chồng cũ bà h


In [176]:
if not os.path.exists(os.path.join(path, 'val.csv')):
    print('val saved')
    val.to_csv(os.path.join(path,'val.csv'), index=False)
# test.to_csv('/home3/phungqv/data_generate/data/test.csv', index=False)
if not os.path.exists(os.path.join(path, 'train.csv')):
    print('train saved')
    train.to_csv(os.path.join(path, 'train.csv'), index=False)


val saved
train saved


## Process human labeled data

In [99]:
f1 = '/home3/cuongld/ASR_team/data_ASR/vietnamese_dataset/transcript_normalize/conversation'
f2 = '/home3/cuongld/ASR_team/data_ASR/vietnamese_dataset/QA/data.csv'

In [130]:
type = ['train', 'valid', 'test']
final_df = pd.DataFrame(columns=['input', 'target'])
for t in type:
    file_path = os.path.join(f1, f'thu_am_{t}.csv')
    norm_path = os.path.join(f1, f'thu_am_{t}.csv.norm')

    fine_df = pd.read_csv(file_path)
    del fine_df['duration']
    fine_df.columns = ['path', 'target']

    dic1 = {'input':[], 'path':[]}
    with open(norm_path) as data:
        first = True
        for line in data:
            if first:
                first = False
                continue
            line = line.split('	')

            dic1['input'].append(line[1])
            dic1['path'].append(line[0])

    temp_df = pd.DataFrame(dic1)
    fine_df = temp_df.merge(fine_df, left_on='path', right_on='path', how='inner')
    del fine_df['path']
    final_df = pd.concat([final_df, fine_df], ignore_index=True)
    print(len(final_df))

final_df = final_df.applymap(lambda s : s.strip())


4465
5353
6317


In [131]:
if not os.path.exists('/home3/phungqv/data_generate/data/finetune/raw/asr_slu.csv'):
    print('asr_slu exported')
    final_df.to_csv('/home3/phungqv/data_generate/data/finetune/raw/asr_slu.csv', index=False)

asr_slu exported


In [125]:
dic2 = {'input':[], 'target':[]}
with open(f2) as data:
    first = True
    for line in data:
        if first:
            first = False
            continue
        line = line.split('	')
        try:
            dic2['input'].append(line[2])
            dic2['target'].append(line[1])
        except:
            print(line)
            break


In [133]:
qna_df = pd.DataFrame(dic2)
qna_df = qna_df.applymap(lambda s : s.strip())

In [136]:
if not os.path.exists('/home3/phungqv/data_generate/data/finetune/raw/qna.csv'):
    print('qna exported')
    qna_df.to_csv('/home3/phungqv/data_generate/data/finetune/raw/qua.csv', index=False)

qna exported


In [142]:
asr_slu = pd.read_csv('/home3/phungqv/data_generate/data/finetune/raw/asr_slu.csv')
qna = pd.read_csv('/home3/phungqv/data_generate/data/finetune/raw/qua.csv')

In [143]:
asr_slu.head()

,input,target
0,em đang sử dụng nhóm thẻ ji xê bê và hình thức...,em đang sử dụng nhóm thẻ jcb và hình thức thẻ ...
1,tuyệt vời quá cảm ơn em nhé chúc em một buổi t...,tuyệt vời quá cảm ơn em nhé chúc em một buổi t...
2,cảm ơn bạn,cảm ơn bạn
3,bên mình có hình thức vay thế chấp vay tín chấ...,bên mình có hình thức vay thế chấp vay tín chấ...
4,chào bạn mình muốn chạy thử tính năng dịch vụ ...,chào bạn mình muốn chạy thử tính năng dịch vụ ...


In [144]:
qna.head()

,input,target
0,a lô một hai ba,alo 1 2 3
1,mình cần hỏi ai để được tư vấn về vấn đề an to...,mình cần hỏi ai để được tư vấn về vấn đề an to...
2,au đi ô này hỏng này,audio này hỏng này
3,mếch in việt nam là gì ạ,make in Vietnam là gì ạ
4,bạn chưa rõ đâu là sự khác nhau giữa chuyển đổ...,bạn chưa rõ đâu là sự khác nhau giữa chuyển đổ...


In [145]:
fine_tune_df = pd.concat([asr_slu, qna], ignore_index=True)

In [147]:
fine_tune_df.head()

,input,target
0,em đang sử dụng nhóm thẻ ji xê bê và hình thức...,em đang sử dụng nhóm thẻ jcb và hình thức thẻ ...
1,tuyệt vời quá cảm ơn em nhé chúc em một buổi t...,tuyệt vời quá cảm ơn em nhé chúc em một buổi t...
2,cảm ơn bạn,cảm ơn bạn
3,bên mình có hình thức vay thế chấp vay tín chấ...,bên mình có hình thức vay thế chấp vay tín chấ...
4,chào bạn mình muốn chạy thử tính năng dịch vụ ...,chào bạn mình muốn chạy thử tính năng dịch vụ ...


In [177]:
def process_fine_tune(s):
    number = re.compile(r'(0|1|2|3|4|5|6|7|7|8|9)+')

    def num_to_word(match):
        result = vietnam_number.n2w(match.group(0)) + ' '
        return result.replace('lẽ', 'linh')
    
    s = s.strip()
    s = s.replace('$', ' đô la ')
    s = s.replace(',', ' phẩy ')
    s = s.replace('%', ' phần trăm ')
    s = s.replace('@', ' a còng ')
    s = s.replace('+' , ' cộng ')

    s = re.sub(number, num_to_word, s)      #for converting numbers

    s = s.translate(str.maketrans(punctuation, ' '*len(punctuation)))
    s = ' '.join(s.split())

    return s
    

In [179]:
fine_tune_df = fine_tune_df.applymap(process_fine_tune)

In [181]:
fine_tune_df.to_csv('/home3/phungqv/data_generate/data/finetune/number_converted/processed.csv', index=False)

In [18]:
df = pd.read_csv('/home3/thanhpv/generated.csv')

In [19]:
del df['path']

In [20]:
df.head(10)

,goc,infer_ra
0,đó xong rồi dần dần rồi bắt đầu qua lại,đó xong mà dần dần rồi mới là qua lại
1,à chị nghĩ là cái món này à cái việc nó cũng n...,à trỷ nghĩ là cái món này ờ cái vị nó cũng ngo...
2,tổng cái số giờ bay theo chúng tôi thống kê đư...,tổng cái số giờ bay theo chúng tôi thống kê đư...
3,cái thứ hai á là cái quy định hiện nay là,cái thứ hai á là cái quy định hiện nay là
4,bạn cũng đề nghị là ban biên tập có thư gửi bà...,bạn cũng đề nghị là ban biên tập có thư gửi bà...
5,giúp chúng ta biết kiểm soát được đồng tiền mà...,chức chúng ta biết kiểm soát được đồng tiền mà...
6,từ đầu chương trình đến giờ thì nói đến hai ng...,từ đầu chương trình đến giờ thì nói đến hai ng...
7,thế là là là là cái thương hiệu cho công ty có...,ý là là là là cái thương hiệu cho công ty của ...
8,cái tôi cái cá nhân trong thiết kế của các bạn...,cái tôi cái cá nhân trong thiết kế của các bạn...
9,mặc dù xét về tính chuyên nghiệp những đầu bếp...,mặc dù xét về tiếng chuyên nghiệp những đầu bế...


In [21]:
df.columns = ['target', 'input']

In [22]:
df.head()

,target,input
0,đó xong rồi dần dần rồi bắt đầu qua lại,đó xong mà dần dần rồi mới là qua lại
1,à chị nghĩ là cái món này à cái việc nó cũng n...,à trỷ nghĩ là cái món này ờ cái vị nó cũng ngo...
2,tổng cái số giờ bay theo chúng tôi thống kê đư...,tổng cái số giờ bay theo chúng tôi thống kê đư...
3,cái thứ hai á là cái quy định hiện nay là,cái thứ hai á là cái quy định hiện nay là
4,bạn cũng đề nghị là ban biên tập có thư gửi bà...,bạn cũng đề nghị là ban biên tập có thư gửi bà...


In [23]:
df = df.sample(frac = 1).reset_index(drop=True)


In [24]:
df.head()

,target,input
0,cũng sẽ cố gắng tạo thêm điều kiện,cũng sẽ cố gắng tạo thêm điều kiện
1,ờ đồng ý nhưng mà không biết phải nói như thế ...,ờ đồng ý nhưng mà không biết phải nói như thế ...
2,chiều nếu anh sắp xếp thời gian cho em á anh sẽ,chiều nếu anh sắp xếp thời gian cho em á anh sẽ
3,em bé ngậm hai viên kẹo và,em bé nhẳn hai diêng kẹo và
4,nhưng khi đụng phải mẫu anh và hắn đã có một n...,nhưng khi đụ phải máu anh và hàng thủ đã có mộ...


In [26]:
val = df.iloc[0:2048]
test = df.iloc[2048:7000]
train = df.iloc[7000:]

In [27]:
val.to_csv('/home3/phungqv/data_generate/data/finetune/val.csv', index=False)
test.to_csv('/home3/phungqv/data_generate/data/finetune/test.csv', index=False)
train.to_csv('/home3/phungqv/data_generate/data/finetune/train.csv', index=False)

In [28]:
train = pd.read_csv('/home3/phungqv/data_generate/data/finetune/train.csv')

In [29]:
train.head()

,target,input
0,các ông bố đi quanh phố cổ và mời bà con du kh...,các ông bố đi quanh phố cổ và mời bà con du kh...
1,sống như vợ chồng hẳn,sống như vợ chồng ở hà nội
2,trở nổi luôn hả dạ có bao giờ gặp ông nào xỉn ...,tử nổi luôn hảdạ có bao giờ gặp ông nào xỉn chưa
3,là một người việt sống ở một nước tư bản nơi m...,là một người việt sống ở một nước tư bản nơi m...
4,gạch lót bị tróc từng mảng kéo dài suốt con đường,gạch loót bị trốc từng mãng kéo dài suốt con đ...


In [3]:
import pandas as pd

In [2]:
val = pd.read_csv('/home3/phungqv/data_generate/data/finetune/val.csv')

In [3]:
val.head()

,target,input
0,cũng sẽ cố gắng tạo thêm điều kiện,cũng sẽ cố gắng tạo thêm điều kiện
1,ờ đồng ý nhưng mà không biết phải nói như thế ...,ờ đồng ý nhưng mà không biết phải nói như thế ...
2,chiều nếu anh sắp xếp thời gian cho em á anh sẽ,chiều nếu anh sắp xếp thời gian cho em á anh sẽ
3,em bé ngậm hai viên kẹo và,em bé nhẳn hai diêng kẹo và
4,nhưng khi đụng phải mẫu anh và hắn đã có một n...,nhưng khi đụ phải máu anh và hàng thủ đã có mộ...


In [1]:
from jiwer import wer

ground_truth = "một hai ba"
hypothesis = "một hai ba"

error = wer(ground_truth, hypothesis)


In [4]:
test = pd.read_csv('/home3/phungqv/data_generate/data/finetune/test.csv')

In [16]:
test.head()

,target,input
0,con nhớ ra có công việc phải làm ở nhà,cô nhớ ra có công việc phải làm ở nhà
1,hiện nay dạ mà à bộ đã đề ra từ năm hai ngàn k...,hiện nay dạ mà à bộ đã để ra từ năm hai ngàn k...
2,sắp đặt rất là hay là mỗi người đều có một vẻ ...,sắp đặt rất là hay là mỗi người đều có một vẻ ...
3,em tặng anh là cái áo màu đỏ anh có bị mất trí...,em tặng anh là cáo màu đỏanh đâu có bị mất trí...
4,quá trình này còn làm ảnh hưởng trực tiếp tới ...,quá trình này còn làm ảnh hưởng trực tiếp tới ...


In [17]:
len(test)

4952

In [18]:
model_wer = []
for i, row in test.iterrows():
    model_wer.append(wer(row.target, row.input))

In [19]:
len(model_wer)

4952

In [20]:
test['model_wer'] = model_wer

In [21]:
test.head()

,target,input,model_wer
0,con nhớ ra có công việc phải làm ở nhà,cô nhớ ra có công việc phải làm ở nhà,0.100000
1,hiện nay dạ mà à bộ đã đề ra từ năm hai ngàn k...,hiện nay dạ mà à bộ đã để ra từ năm hai ngàn k...,0.047619
2,sắp đặt rất là hay là mỗi người đều có một vẻ ...,sắp đặt rất là hay là mỗi người đều có một vẻ ...,0.000000
3,em tặng anh là cái áo màu đỏ anh có bị mất trí...,em tặng anh là cáo màu đỏanh đâu có bị mất trí...,0.333333
4,quá trình này còn làm ảnh hưởng trực tiếp tới ...,quá trình này còn làm ảnh hưởng trực tiếp tới ...,0.000000


In [24]:
sum(test.model_wer)/len(test.model_wer) * 100

10.491930527618326

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
pretrained = AutoModelForSeq2SeqLM.from_pretrained('/home3/phungqv/data_generate/text-normalization/checkpoint-84561')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
import time
device = "cuda:0"
pretrained = pretrained.to(device)


In [41]:
import tqdm

In [40]:
norm_sentence = []
norm_wer = []
norm_time = []
for i, row in test.iterrows():
    start_time = time.time()
    inputs = tokenizer.encode(row.input, return_tensors='pt')
    inputs = inputs.to(device)
    outputs = pretrained.generate(inputs, max_length=128, num_beams=6, early_stopping=True)
    outputs = outputs.tolist()
    sentence = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    norm_time.append(time.time() - start_time)
    norm_sentence.append(sentence)
    norm_wer.append(wer(row.target, sentence))


In [44]:
test['norm_sentence'] = norm_sentence

In [46]:
test['norm_wer'] = norm_wer

In [47]:
test.head()

,target,input,model_wer,norm_sentence,norm_wer
0,con nhớ ra có công việc phải làm ở nhà,cô nhớ ra có công việc phải làm ở nhà,0.100000,cô nhớ ra có công việc phải làm ở nhà,0.100000
1,hiện nay dạ mà à bộ đã đề ra từ năm hai ngàn k...,hiện nay dạ mà à bộ đã để ra từ năm hai ngàn k...,0.047619,hiện nay dạ mà à bộ đã để ra từ năm 2 ngàn 201...,0.238095
2,sắp đặt rất là hay là mỗi người đều có một vẻ ...,sắp đặt rất là hay là mỗi người đều có một vẻ ...,0.000000,sắp đặt rất là hay là mỗi người đều có một vẻ ...,0.000000
3,em tặng anh là cái áo màu đỏ anh có bị mất trí...,em tặng anh là cáo màu đỏanh đâu có bị mất trí...,0.333333,em tặng anh là cáo màu đón anh đâu có bị mất t...,0.333333
4,quá trình này còn làm ảnh hưởng trực tiếp tới ...,quá trình này còn làm ảnh hưởng trực tiếp tới ...,0.000000,quá trình này còn làm ảnh hưởng trực tiếp tới ...,0.000000


In [50]:
test['norm_time'] = norm_time

In [54]:
sum(test.norm_wer) / len(test.norm_wer)

0.12478387269263236

In [56]:
test.to_csv('/home3/phungqv/data_generate/wer/wer.csv',index=False)

IsADirectoryError: [Errno 21] Is a directory: '/home3/phungqv/data_generate/wer/'

In [74]:
better = test[test.model_wer - test.norm_wer > 0.1]
len(better)

96

In [75]:
for i in range(50):
    row = better.iloc[i]
    print('ground truth: ' + row['target'] )
    print('asr ouput:    ' + row['input'])
    print('normalized:   ' + row['norm_sentence'])
    print('\n')

ground truth: bảo tao xác nhận nằm ngửa ra nhà bảo tao xác nhận ui giời ôi
asr ouput:    tà tao sác nhệ ước á nằm ngửa ra nhà mà tao xác nhậ ú rồi i
normalized:   tà tao xác nghệ nước á nằm ngửa ra nhà mà tao xác nhận ú rồi i


ground truth: chỉ cần đưa ra một vài dẫn chứng thôi
asr ouput:    chỉ cần đưa ra một vài dẫn trứng thôi
normalized:   chỉ cần đưa ra một vài dẫn chứng thôi


ground truth: nó chẳng ảnh hưởng đến vụ tiếp theo
asr ouput:    nó chẽ ảnh hưởng đến bộ tiếp deo
normalized:   nó chẽ ảnh hưởng đến bộ tiếp theo


ground truth: buôn tiền qua mạng nở rộ
asr ouput:    bun tiền qua mạng nở rộ
normalized:   buôn tiền qua mạng nở rộ


ground truth: có người ngỏ lời cầu hôn
asr ouput:    có người ngò lời cầu hôn
normalized:   có người ngỏ lời cầu hôn


ground truth: rất lớn trong cái việc thành công trong lĩnh vực giáo dục nghề nghiệp trong thời gian vừa qua
asr ouput:    rất lớn trong cái việc thành công trong lĩnh vựcgiáo dục nghiệp trong thời gian vừa qua
normalized:   rất lớ

In [76]:
worse = test[test.model_wer - test.norm_wer < -0.1]

In [77]:
len(worse)

423

In [78]:
for i in range(50):
    row = worse.iloc[i]
    print('ground truth: ' + row['target'] )
    print('asr ouput:    ' + row['input'])
    print('normalized:   ' + row['norm_sentence'])
    print('\n')

ground truth: hiện nay dạ mà à bộ đã đề ra từ năm hai ngàn không trăm mười dạ thưa bác sĩ à
asr ouput:    hiện nay dạ mà à bộ đã để ra từ năm hai ngàn không trăm mười dạ thưa bác sĩ à
normalized:   hiện nay dạ mà à bộ đã để ra từ năm 2 ngàn 2010 dạ thưa bác sĩ à


ground truth: nhất là cái giọng rao chè như hát
asr ouput:    nhất là cái giọng rao chẻ như hát
normalized:   nhất là cái giọng dao trẻ như h


ground truth: hơi xanh bủng do bị ướm nắng
asr ouput:    hơi xanh bủng do bị ướm nắng
normalized:   hơi xanh chủng do bị gươm nắng


ground truth: ờ vậy là ba mẹ nhưng mà ba mẹ em có kiểm soát không ba mẹ chỉ tịch thu điện thoại thôi chứ
asr ouput:    ờ vậy là ba mẹ nhưng mà ba mẹ kông có kiểm soát không ba mẹ chỉ tịch thuơng điện thoại thôi chưa
normalized:   ở vậy là 3 mẹ nhưng mà 3 mẹ không có kiểm soát không 3 mẹ chỉ tịch tổng điện thoại thôi chưa


ground truth: chính nghiệp khóa mười ba đã tiến hành phiên họp toàn thể lần thứ tư
asr ouput:    chính hiệp khoá mười ba đã tiến hành